### Import Packages

Here is where we import all the packages we need. Noteably, NumPy is a standard computing package, Pandas is for data manipulation, and Torch is the Python machine learning package.

In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch as torch
import torch.nn as nn
import os
from math import *
import copy
from scipy.integrate import odeint
from matplotlib.lines import Line2D
from tabulate import tabulate
import matplotlib.cm as cm
import matplotlib
import random
from mpl_toolkits.mplot3d import Axes3D
from math import *
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
from scipy.interpolate import UnivariateSpline
from torch.utils.data import DataLoader, Dataset
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu") 
#this line says that if we have a GPU, it will do the computations there.
print(device)

cpu


### Formatting .csv data files

In [32]:
#upload .csv file and skip 13 header rows
DTS_calib = pd.read_csv('00 to 23 31Jan2020 copy.csv',skiprows=range(13))
#stop after 100 depths (for this early version)
DTS_calib = DTS_calib.head(100)
#remove column of depths
DTS_calib = DTS_calib.iloc[:,1:]

#check to see the rows and columns have been skipped
print(DTS_calib.head())

#create new file which stores the updated version (without headers or depth)
DTS_calib.to_csv('00 to 23 31Jan2020 no header top 100.csv', index=False)


   21.5799    21.54    21.44  21.5799.1    21.51    21.42  21.4799    21.34  \
0    21.51  21.3199  21.2399    21.3700  21.4699  21.3899  21.1800  21.2099   
1    28.69  28.4799  28.4099    28.5699  28.6599  28.5599  28.3199  28.2900   
2    21.11  20.9599  21.0400    21.0400  21.0799  20.9899  20.8700  20.8099   
3    21.12  21.0300  21.1200    21.0599  21.1000  20.9500  20.9599  20.8700   
4    21.17  21.1800  21.1900    21.1599  21.3400  21.1499  21.1200  20.9599   

     21.43    21.35  ...  21.2299  21.2199    21.26    21.25  21.2099  \
0  21.2099  21.2900  ...  21.1399  21.1200  21.2099  21.1499  21.0900   
1  28.3999  28.3500  ...  28.0799  28.2399  28.3999  28.2299  28.0900   
2  20.9099  20.7600  ...  20.3799  20.7099  20.9400  20.7000  20.4899   
3  20.9400  20.8299  ...  20.5000  20.7199  20.9899  20.7500  20.6399   
4  20.9400  20.9400  ...  20.7500  20.7800  21.0799  20.8700  20.9400   

   21.25.1     21.2  21.5499  21.42.1  21.35.1  
0  21.1299  21.0900  21.3700  21.2800

In [8]:
#convert the DataFrame which we created using Pandas from .csv file into a NumPy array
observed_temp = DTS_calib.to_numpy()

#Define the dimensions of the array
num_depths, num_hours = observed_temp.shape

#define a time vector (to be used in later versions to compute splines, this is currently arbitrary)
t = np.linspace(0, 23, num = 24)
print(t)

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23.]


### Spline approximation

In [37]:
#initialize arrays to store the spline-smoothed data and derivatives
smoothed_observed_temp = np.zeros(observed_temp.shape)
spline_deriv=np.zeros(observed_temp.shape)

# Loop through each row (depth) and fit a spline to smooth the temperature data
for i in range(observed_temp.shape[0]):
    # Fit a spline to the data for each row (depth)
    spline = UnivariateSpline(t, observed_temp[i, :], s=0.08)

    # Evaluate the spline at each hour to get the smoothed temperature data
    smoothed_observed_temp[i, :] = spline(t)

    # Calculate the spline derivative at each hour for the given row (depth)
    spline_deriv[i, :] = spline.derivative()(t)

##Plot the smoothed temp curve and the spline derivatives!!

### Difference quotients to get derivative approximation

In [38]:
#define the time step size (delta_t)
delta_t=[]
for i in range(len(t) -1):
    diff = t[i+1] - t[i]
    delta_t.append(diff)

print(delta_t)

#initialize array to store the difference quotients 
diff_quots=np.zeros((num_depths, num_hours-1))
    
#compute difference quotient for each row (because we want the difference quotient hour to hour)
#using smothed_observed_data because the purpose of the spline approximation is to get better difference quotients
for i in range(num_depths):
    for j in range(num_hours - 1):
        diff_quots[i,j] = (smoothed_observed_temp[i, j+1] - smoothed_observed_temp[i,j])/delta_t[j]
        
print(diff_quots)

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[[-0.27981015  0.05272412  0.12292976 ...  0.07957745  0.1018994
  -0.1760312 ]
 [-0.31483118  0.08191411  0.15308918 ...  0.10629303  0.06108875
  -0.04475161]
 [ 0.01107166 -0.01170817 -0.02708155 ...  0.11057456  0.13815104
   0.13818584]
 ...
 [ 0.07013962  0.02320785  0.03510411 ...  0.00053649  0.01672686
   0.03811482]
 [ 0.06260708  0.03455807  0.04193977 ... -0.0048157   0.00271191
   0.01316791]
 [ 0.06027059  0.03243285  0.05230167 ...  0.00321359  0.01505309
   0.03066906]]


### Setup the model

In [13]:
from torch.autograd import Variable
from torch.autograd import Function # import Function to create custom activations
from torch.nn.parameter import Parameter # import Parameter to create custom activations with learnable parameters

In [39]:
#this block is neural network specific things we want to load in order to build and train the model

class LoadDataset(Dataset):
    def __init__(self, inputs, targets):
        self.x = inputs
        self.y = targets

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        inputs = self.x[idx, :]
        targets = self.y[idx, :]

        return inputs, targets

In [34]:
#here we define a few hyperparameters
epochs = 200
batch_size = 100
input_size = 1 #we give the net as input the temperature at a fixed time and depth
hidden_size1 = 10
hidden_size2 = 20
output_size = 1 #the net outputs the T' for the given T. This gives one equation for all depths

#to have one equation per depth, the network should have input 100
# and output 100 where 100 = number of depths at a fixed time (this is for a later version)

class Net(nn.Module) :

    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(Net, self).__init__()
        self.linear1 = torch.nn.Linear(input_size,   hidden_size1, bias=True)
        self.linear2 = torch.nn.Linear(hidden_size1, hidden_size2, bias=True)
        self.linear3 = torch.nn.Linear(hidden_size2, hidden_size2, bias=True)
        self.linear4 = torch.nn.Linear(hidden_size2, hidden_size2, bias=True)
        self.linear5 = torch.nn.Linear(hidden_size2, hidden_size2, bias=True)
        self.linear6 = torch.nn.Linear(hidden_size2, hidden_size2, bias=True)
        self.linear7 = torch.nn.Linear(hidden_size2, hidden_size2, bias=True)
        self.linear8 = torch.nn.Linear(hidden_size2, output_size,  bias=True)

        self.mylrelu = torch.nn.LeakyReLU()    #Two_corners_LReLU()
        
    def forward(self, x):
        x = self.mylrelu(self.linear1(x))
        x = self.mylrelu(self.linear2(x))
        x = self.mylrelu(self.linear3(x))
        x = self.mylrelu(self.linear4(x))
        x = self.mylrelu(self.linear5(x))
        x = self.mylrelu(self.linear6(x))
        x = self.mylrelu(self.linear7(x))
        x = self.linear8(x)
        return x
        
#name our network net
net = Net(input_size, hidden_size1, hidden_size2, output_size).to(device)
print(net)

Net(
  (linear1): Linear(in_features=1, out_features=10, bias=True)
  (linear2): Linear(in_features=10, out_features=20, bias=True)
  (linear3): Linear(in_features=20, out_features=20, bias=True)
  (linear4): Linear(in_features=20, out_features=20, bias=True)
  (linear5): Linear(in_features=20, out_features=20, bias=True)
  (linear6): Linear(in_features=20, out_features=20, bias=True)
  (linear7): Linear(in_features=20, out_features=20, bias=True)
  (linear8): Linear(in_features=20, out_features=1, bias=True)
  (mylrelu): LeakyReLU(negative_slope=0.01)
)


### Training the Network

In [47]:
#define our inputs and targets for the training data
train_data = torch.from_numpy(observed_temp)
train_expected = torch.from_numpy(diff_quots)

#define dataset to use when training
dataset = LoadDataset(train_data, train_expected)

# This loads data in batches which you can now change to be more than 1 without errors
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
#we use Adam as the optimizer, an automatic stochastic gradient descent algorithm
#we can tune the hyperparameters if desired
optimizer = torch.optim.Adam(net.parameters(), lr=1e-03, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

#define the loss function, in our case, mean squared error
loss = torch.nn.MSELoss()

#initialize an empty list of losses, loss starts at 0.
train_loss = 0
l2_loss_list = []
net.train()
for epoch in range(epochs): 
    # if epoch % 10 == 0:
    #     for param_group in optimizer.param_groups:
    #         param_group['lr'] /= 2.

    batch_idx = 0
    for input_data, targets in dataloader:
        input_data = input_data.to(device) #or use batch size instead of -1 but add , drop_last=True in DataLoader
        preds = net(input_data)
        targets = targets.to(device)
        lo = loss(preds, targets)
        train_loss += lo.detach().numpy()        
        optimizer.zero_grad()
        lo.backward()
        optimizer.step()
        batch_idx += 1
        
    l2_loss_list.append(train_loss/batch_idx)
    if epoch%1 ==0:
        print('Train Loss = ', l2_loss_list[-1])

RuntimeError: mat1 and mat2 shapes cannot be multiplied (100x24 and 1x10)

### Visualizing output

Differential equation reconstruction

In [51]:
# Initial values, each column is a vector x0 of initial conditions
components = 1 #means each observation (vector) has one component, ie only one variable is being modelled (temperature)
n_initial_cond = 100 #100 initial conditions for each component - 100 depths


#compute solution
time_steps_sol = len(t) #stores the number of time steps in the solution

#initialize 3D array of zeroes to store the true solutions to the ODE system for different initial conditions and time steps
true_solut = np.zeros((n_initial_cond,components,time_steps_sol))

#initialize 3D array of zeroes to store the network solutions to the ODE system for different initial conditions and time steps
net_solut = np.zeros((n_initial_cond,components,time_steps_sol))


#loop set up to iterate i from 0 to (n_initial_cond-1). 
#This means that the loop will run 100 times in this case since n_initial_cond is 100.

for i in range(n_initial_cond): #number of inputs
    # Bundle initial conditions for ODE solver
    y0 = in_cond[:, i]
    # Call the ODE solver
    psoln = np.transpose(odeint(f_net, y0, t))
    net_solut[i,:,:] = psoln
net_solut = net_solut.reshape(n_initial_cond,time_steps_sol)

NameError: name 'in_cond' is not defined